# Importing data from databases (Part 1)

Many companies store their information in relational databases. The R community has also developed R packages to get data from these architectures. Connecting to a database and retrieve data from it.

## Connect to a database

### Establish a connection

The first step to import data from a SQL database is creating a connection to it.

`dbConnect()` creates a connection between R session and a SQL database. The first argument has to be a `DBIdriver` object, that specifies how connections are made and how data is mapped between R and the database. Specifically for MySQL databases, you can build such a driver with `RMySQL::MySQL()`.

If the MySQL database is a remote database hosted on a server, you'll also have to specify the following arguments in `dbConnect():` `dbname`, `host`, `port`, `user` and `password`. 

In [7]:
library("DBI")
library("RMySQL")
con <- dbConnect(RMySQL::MySQL(), 
                 dbname = "tweater", 
                 host = "courses.csrrinzqubik.us-east-1.rds.amazonaws.com", 
                 port = 3306,
                 user = "student",
                 password = "datacamp")
con

<MySQLConnection:0,2>

## Import tables

### List the database tables

After you've successfully connected to a remote MySQL database, the next step is to see what tables the database contains. You can do this with the `dbListTables()` function. This function requires the connection object as an input, and outputs a character vector with the table names.

In [12]:
tables_names = dbListTables(con)
str(tables_names)

 chr [1:3] "comments" "tweats" "users"


`dbListTables()` can be very useful to get a first idea about the contents of your database

### Import users

The database contains data on a more tasty version of Twitter, namely Tweater. Users can post tweats with short recipes for delicious snacks. People can comment on these tweats. There are three tables: **`users`, `tweats`, and `comments`** that have relations among them.

To importing the data on the users into R session, you use `dbReadTable()` function. Pass it the connection object `(con)`, followed by the name of the table you want to import. The resulting object is a standard R data frame.

In [13]:
tables = lapply(tables_names, dbReadTable, conn=con)
tables

id,tweat_id,user_id,message
1022,87,7,nice!
1000,77,7,great!
1011,49,5,love it
1012,87,1,awesome! thanks!
1010,88,6,yuck!
1026,77,4,not my thing!
1004,49,1,this is fabulous!
1030,75,6,so easy!
1025,88,2,oh yes
1007,49,3,serious?


# Importing data from databases (Part 2)

Importing an entire table from a database while you might only need a tiny bit of information seems like a lot of unncessary work. SQL queries help you make things more efficient by performing some computations on the database side.

## SQL Queries inside R

### Query tweater (1)

 If you want to do some analyses on data, it's possible that you only need a fraction of this data. In this case, it's a good idea to send SQL queries to your database, and only import the data you actually need into R.
 
 `dbGetQuery()` is what you need. You first pass the connection object to it. The second argument is an SQL query in the form of a character string.

In [14]:
elisabeth = dbGetQuery(con, "SELECT tweat_id FROM comments WHERE user_id = 1")
elisabeth

tweat_id
87
49
77
77


### Query tweater (2)

tweats where date is higher than '2015-09-21'

In [15]:
latest = dbGetQuery(con, "SELECT post FROM tweats WHERE date > '2015-09-21'")
latest

post
open and crush avocado. add shrimps. perfect starter.
"nachos. add tomato sauce, minced meat and cheese. oven for 10 mins."
just eat an apple. simply and healthy.


### Query tweater (3)

message column from the comments table where the tweat_id is 77 and the user_id is greater than 4.

In [16]:
specific = dbGetQuery(con, "SELECT message FROM comments WHERE tweat_id=77 AND user_id>4")
specific

message
great!


### Query tweater (4)

`id` and `name` columns from the `users` table where the number of characters in the `name` is strictly less than `5`.

In [17]:
short = dbGetQuery(con, "SELECT id, name FROM users WHERE CHAR_LENGTH(name) < 5")
short

id,name
2,mike
3,thea
6,kate


### Join the query madness!



In [22]:
dbGetQuery(con,"SELECT name, post 
FROM users 
INNER JOIN tweats 
ON users.id = user_id 
WHERE date > '2015-09-19'")

name,post
elisabeth,"nachos. add tomato sauce, minced meat and cheese. oven for 10 mins."
oliver,open and crush avocado. add shrimps. perfect starter.
kate,2 slices of bread. add cheese. grill. heaven.
anjali,just eat an apple. simply and healthy.


In [24]:
dbGetQuery(con, "SELECT post, message
  FROM tweats INNER JOIN comments on tweats.id = tweat_id
    WHERE tweat_id = 77")

post,message
2 slices of bread. add cheese. grill. heaven.,great!
2 slices of bread. add cheese. grill. heaven.,not my thing!
2 slices of bread. add cheese. grill. heaven.,couldn't be better
2 slices of bread. add cheese. grill. heaven.,saved my day


## DBI internals

### Send - Fetch - Clear

comments for the users with an id above 4.

In [25]:
res <- dbSendQuery(con, "SELECT * FROM comments WHERE user_id > 4")
dbFetch(res, n=2)
dbFetch(res)
dbClearResult(res)

id,tweat_id,user_id,message
1022,87,7,nice!
1000,77,7,great!


id,tweat_id,user_id,message
1011,49,5,love it
1010,88,6,yuck!
1030,75,6,so easy!


[1] TRUE

### Be polite and ... `dbDisconnect()`

post and date columns from the observations in tweats where the character length of the post variable exceeds 40.

In [27]:
long_tweats = dbGetQuery(con, "SELECT post, date FROM tweats WHERE CHAR_LENGTH(post) > 40")
long_tweats
dbDisconnect(con)

post,date
wash strawberries. add ice. blend. enjoy.,2015-09-14
2 slices of bread. add cheese. grill. heaven.,2015-09-21
open and crush avocado. add shrimps. perfect starter.,2015-09-22
"nachos. add tomato sauce, minced meat and cheese. oven for 10 mins.",2015-09-22


[1] TRUE